# Installations

In [1]:
# %pip install "opencv-python-headless<4.3"
# %pip install cellpose
# %pip install cytoself

# %pip install adjustText

# %pip install --upgrade scikit-image

# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random
from model import load_data, get_analytics
from cytoself_custom import plot_umap, calc_umap_embvec, plot_feature_spectrum_from_image
from synthetic_multiplexing import multiplex
from metrics import plot_boostrapping, calc_bootstrapping, plot_metrics
from explainability import generate_images, plot_clusters_distance_heatmap
from utils import get_colors_dict
from sklearn.metrics.pairwise import euclidean_distances

import config
from config import SEED, PRETRAINED_MODEL_PATH, MODEL_PATH, METRICS_FOLDER, METRICS_MATCH_PATH, METRICS_RANDOM_PATH


from metrics import calc_reconstruction_error
import os


np.random.seed(SEED)
random.seed(SEED)

%reload_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
import logging
log_file_path = f"./logs/eval_log.log"
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler()
    ])

logging.info("init")

2023-01-02 16:43:06 INFO init


In [4]:
batch1 = ["./data/processed/220629_neurons_12markers_unstressed",
          "./data/processed/220707_neurons_12markers_stressed"]

batch2 = ["./data/processed/220814_neurons",
            "./data/processed/220818_neurons",
            "./data/processed/220831_neurons",
            "./data/processed/220908",
            "./data/processed/220914"]

model_path18 = os.path.join("./models", "MODEL18_model_weights.0040.h5")
# model_path19 = os.path.join("./models", "MODEL19_model_weights.0018.h5") #MODEL_PATH
# model_path20 = os.path.join("./models", "MODEL20_model_weights.0017.h5") #MODEL_PATH
# model_path_1p2 = os.path.join("./models", "MODEL1p2_model_weights.0011.h5") #MODEL_PATH
# model_path21 = os.path.join("./model_outputs", "model_weights.0056.sagy.1412.h5")

markers = ["CD41", "CLTC", "FMRP", "G3BP1", "KIF5A", "NONO", "Phalloidin",\
            "PSD95", "PURA", "SQSTM1", "TDP43", "TIA1", "NEMO", "DCP1A",\
                "TOMM20", "ANXA11", "Calreticulin", "FUS", "LAMP1",\
                    "mitotracker", "Nucleolin", "SNCA",\
                        "GM130", "PEX14", "PML", "DAPI"] #Without "lysotracker" and "Syto12"

# markers = ['G3BP1']
condition_l=True
type_l=True
conds_include = ["unstressed", "stressed"]
cell_lines_include = ['WT', 'TDP43', 'FUS', 'OPTN', 'TBK1']

logging.info(f"Markers: {markers}; Conditions: {conds_include}; Cell lines: {cell_lines_include}")

# images_indexes = random.sample(range(10), 10)
for batch_name, input_folders in zip(["Batch2"], [batch2]):#zip(["Batch2", "Batch1"], [batch2, batch1]):
    # Load data
    logging.info(f"[{batch_name}] Load data")
    if batch_name == 'Batch2':
        images, labels, labels_changepoints, markers_order = load_data(input_folders, markers=markers, \
                                                                    condition_l=condition_l, type_l=type_l,\
                                                                        split_by_set=True,
                                                                        set_type = 'test',
                                                                        conds_include=conds_include,
                                                                        cell_lines_include=cell_lines_include,
                                                                        split_by_set_include_batch=[("WT", "unstressed")])
    # else:
    #     images, labels, labels_changepoints, markers_order = load_data(input_folders, markers=markers, \
    #                                                                 condition_l=condition_l, type_l=type_l,\
    #                                                                     split_by_set=False,
    #                                                                     conds_include=conds_include,
    #                                                                     cell_lines_include=cell_lines_include)
    labels_s = pd.Series(labels.reshape(-1,))

    # for name, model_path in zip(['19', '20', 'pretext'], [model_path19, model_path20, model_path_1p2]):
    for model_name, model_path in zip(['MODEL18'], [model_path18]):
        # Load models
        # logging.info(f"[{batch_name} - {model_name}] Get analytics")
        # analytics = get_analytics(images, labels, model_path=model_path)


        for m in ["TDP43", "FUS"]:
            logging.info(f"[{batch_name} - {model_name}] Marker: {m}")
            # markers_indexes = labels_s[labels_s.str.contains(f'{m}_')].index
            markers_indexes = labels_s[labels_s.str.startswith(f'{m}_')].index
            images_subset = images[markers_indexes]
            labels_subset = labels[markers_indexes]
            logging.info(f"[{batch_name} - {model_name} - {m}] Get analytics")
            analytics_marker = get_analytics(images_subset, labels_subset, model_path=model_path)
            # colors_dict = get_colors_dict(labels_subset)
            logging.info(f"[{batch_name} - {model_name}] Eval")
            rec_errors = calc_reconstruction_error(analytics_marker, images_indexes=np.arange(len(images_subset)), entire_data=images, show=False)
            logging.info(f"[{batch_name} - {model_name}] Rec errors {rec_errors}")
            # logging.info(f"[{batch_name} - {model_name}] Rec errors mean {np.mean(rec_errors)}")
            
            # umap_vec, _ = plot_umap(analytics_marker, cmap='tab20', s=15,savefig=True, to_annot=False,
            #                          output_filename=f"{model_name}_EVAL_{batch_name}_{m}", titles=f'[{batch_name} - {model_name}] Marker: {m}')

2023-01-02 16:43:06 INFO Markers: ['CD41', 'CLTC', 'FMRP', 'G3BP1', 'KIF5A', 'NONO', 'Phalloidin', 'PSD95', 'PURA', 'SQSTM1', 'TDP43', 'TIA1', 'NEMO', 'DCP1A', 'TOMM20', 'ANXA11', 'Calreticulin', 'FUS', 'LAMP1', 'mitotracker', 'Nucleolin', 'SNCA', 'GM130', 'PEX14', 'PML', 'DAPI']; Conditions: ['unstressed', 'stressed']; Cell lines: ['WT', 'TDP43', 'FUS', 'OPTN', 'TBK1']
2023-01-02 16:43:06 INFO [Batch2] Load data


#########################################################
########### Splitting by set! (test) #############
#########################################################
Input folder: ./data/processed/220814_neurons
Input subfolder: CD41
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CD41/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Input subfolder: CLTC
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CLTC/220809_iNDI_FUS_unstressed-DAPI_Phalloidin_PSD95_CLTC-01_FUS.tif_processed
Input subfolder: DAPI
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_G3BP1_TIA1_KIF5A-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_PURA_SQSTM1_FMRP-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/D

2023-01-02 17:04:19 INFO [Batch2 - MODEL18] Marker: TDP43
2023-01-02 17:04:19 INFO [Batch2 - MODEL18 - TDP43] Get analytics


X, y:
(2241, 100, 100, 2) (2241,)
TDP43_FUS_unstressed      280
TDP43_OPTN_stressed       259
TDP43_WT_unstressed       250
TDP43_OPTN_unstressed     244
TDP43_TBK1_stressed       244
TDP43_FUS_stressed        241
TDP43_TDP43_unstressed    218
TDP43_WT_stressed         178
TDP43_TBK1_unstressed     166
TDP43_TDP43_stressed      161
dtype: int64
Creating the model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2023-01-02 17:04:19 WARNING From /home/labs/hornsteinlab/sagyk/anaconda3/envs/cytoself/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


...

2023-01-02 17:10:35 WARNING From /home/labs/hornsteinlab/Collaboration/MOmaps/metrics.py:144: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2023-01-02 17:10:59 INFO [Batch2 - MODEL18] Rec errors [{'by_self': 0.05588658138065022, 'by_all': 0.06700338341401894, 'unnormalized': 0.00043108381525851577}, {'by_self': 0.1215480474834264, 'by_all': 0.07501877442997555, 'unnormalized': 0.000577187492759567}]
2023-01-02 17:10:59 INFO [Batch2 - MODEL18] Marker: FUS
2023-01-02 17:10:59 INFO [Batch2 - MODEL18 - FUS] Get analytics


X, y:
(2809, 100, 100, 2) (2809,)
FUS_OPTN_unstressed     324
FUS_WT_unstressed       317
FUS_TBK1_unstressed     311
FUS_TBK1_stressed       306
FUS_OPTN_stressed       300
FUS_TDP43_stressed      283
FUS_FUS_unstressed      277
FUS_WT_stressed         263
FUS_TDP43_unstressed    225
FUS_FUS_stressed        203
dtype: int64
Creating the model


All files will be saved at model_outputs/


Loading weights


2023-01-02 17:11:21 INFO [Batch2 - MODEL18] Eval


./models/MODEL18_model_weights.0040.h5
is loaded.
Ground truth:
['FUS_FUS_stressed' 'FUS_FUS_unstressed' 'FUS_OPTN_stressed'
 'FUS_OPTN_unstressed' 'FUS_TBK1_stressed' 'FUS_TBK1_unstressed'
 'FUS_TDP43_stressed' 'FUS_TDP43_unstressed' 'FUS_WT_stressed'
 'FUS_WT_unstressed']
Inferring embedding vectors...
2809/2809 [==============================] - 222s 79ms/sample
(2809, 25, 25, 64) (2809, 12, 12, 64)


2023-01-02 17:19:04 INFO [Batch2 - MODEL18] Rec errors [{'by_self': 0.06199090546099383, 'by_all': 0.09592350473691537, 'unnormalized': 0.0006171489899166198}, {'by_self': 0.11269578614093587, 'by_all': 0.1991137264552743, 'unnormalized': 0.00153196254430961}]


In [5]:
logging.info(f"DONE!")

2023-01-02 17:19:05 INFO DONE!
